In [7]:
import sqlite3
import sys
sys.path.append("..")
from rcn_py import rsd
from rcn_py import orcid

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Create project table, project member table, software table, software contributor table.
rsd.create_rsd_database("/Users/jennifer/RCSNA/data/rsd.db")

'Created'

In [5]:
# Data storage
rsd.store_rsd("/Users/jennifer/RCSNA/data/rsd.db")

'Done'

In [169]:
# a test for reading the database
con = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
cur = con.cursor()

In [170]:
res = cur.execute("SELECT id, title, description FROM project")
project_description = res.fetchall()

In [171]:
cur.close()

In [10]:
proj_clusters, proj_idx2topics = rsd.project_software_cluster("/Users/jennifer/RCSNA/data/rsd.db", 'project')
soft_clusters, soft_idx2topics = rsd.project_software_cluster("/Users/jennifer/RCSNA/data/rsd.db", 'software')

In [12]:
au_group_proj = rsd.author_cluster("/Users/jennifer/RCSNA/data/rsd.db", "project", proj_clusters)
au_group_soft = rsd.author_cluster("/Users/jennifer/RCSNA/data/rsd.db", "software", soft_clusters)

In [13]:
au_group_proj

{'0000-0001-5107-3531': 0,
 '0000-0001-5196-5791': 1,
 '0000-0001-5208-4283': 0,
 '0000-0001-5237-9583': 1,
 '0000-0001-5239-5321': 0,
 '0000-0001-5240-9770': 0,
 '0000-0001-5267-3002': 1,
 '0000-0001-5283-1330': 0,
 '0000-0001-5301-3276': 0,
 '0000-0001-5301-4981': 0,
 '0000-0001-5338-5299': 0,
 '0000-0001-5371-5876': 0,
 '0000-0001-5387-341X': 0,
 '0000-0001-5397-1503': 0,
 '0000-0001-5450-4333': 0,
 '0000-0001-5468-1030': 0,
 '0000-0001-5468-2025': 0,
 '0000-0001-5514-6021': 1,
 '0000-0001-5540-2822': 0,
 '0000-0001-5565-7577': 0,
 '0000-0001-5588-7903': 0,
 '0000-0001-5619-4017': 0,
 '0000-0001-5662-2843': 0,
 '0000-0001-5666-1658': 0,
 '0000-0001-5676-8834': 0,
 '0000-0001-5680-5957': 0,
 '0000-0001-5817-7675': 0,
 '0000-0001-5839-0302': 0,
 '0000-0001-5852-4155': 0,
 '0000-0001-5974-6219': 1,
 '0000-0001-6005-3110': 0,
 '0000-0001-6076-4406': 0,
 '0000-0001-6090-0535': 0,
 '0000-0001-6128-6979': 1,
 '0000-0001-6159-041X': 1,
 '0000-0001-6350-2380': 2,
 '0000-0001-6366-2173': 0,
 

In [15]:
link_proj = rsd.rsd_relationship("/Users/jennifer/RCSNA/data/rsd.db", "project")
link_soft = rsd.rsd_relationship("/Users/jennifer/RCSNA/data/rsd.db", "software")

In [16]:
link_proj

[('0000-0002-0948-1176', '0000-0002-8276-8819'),
 ('0000-0002-0948-1176', '0000-0002-4411-6932'),
 ('0000-0002-8276-8819', '0000-0002-4411-6932'),
 ('0000-0002-0246-8534', '0000-0003-0087-6021'),
 ('0000-0002-0246-8534', '0000-0001-9795-7981'),
 ('0000-0002-0246-8534', '0000-0002-5821-2060'),
 ('0000-0003-0087-6021', '0000-0001-9795-7981'),
 ('0000-0003-0087-6021', '0000-0002-5821-2060'),
 ('0000-0001-9795-7981', '0000-0002-5821-2060'),
 ('0000-0002-6371-2991', '0000-0002-1211-4238'),
 ('0000-0002-6371-2991', '0000-0002-2888-4202'),
 ('0000-0002-6371-2991', '0000-0002-1030-0512'),
 ('0000-0002-6371-2991', '0000-0002-5413-9038'),
 ('0000-0002-6371-2991', '0000-0003-2571-9102'),
 ('0000-0002-6371-2991', '0000-0002-9084-552X'),
 ('0000-0002-6371-2991', '0000-0002-0948-1176'),
 ('0000-0002-6371-2991', '0000-0002-8431-081X'),
 ('0000-0002-1211-4238', '0000-0002-2888-4202'),
 ('0000-0002-1211-4238', '0000-0002-1030-0512'),
 ('0000-0002-1211-4238', '0000-0002-5413-9038'),
 ('0000-0002-1211-42

In [127]:
rsd.build_network_pyvis("/Users/jennifer/RCSNA/data/rsd.db", "project")

'html file is created'

In [128]:
rsd.build_network_pyvis("/Users/jennifer/RCSNA/data/rsd.db", "software")

'html file is created'

In [160]:
rsd.build_network_gephi("/Users/jennifer/RCSNA/data/rsd.db", "project")


'gexf file is created'

In [161]:
rsd.build_network_gephi("/Users/jennifer/RCSNA/data/rsd.db", "software")

'gexf file is created'

## Neo4j

In [15]:
from py2neo import Graph
from neo4j import GraphDatabase
import pandas as pd
import requests
from itertools import combinations

In [5]:
uri = "neo4j+s://a93e1a64.databases.neo4j.io"
user = "neo4j"
password = "ifVwf-BGTZt1ltrNgVbh5Q6aGSxQSksQJSxBUbJb1BE"


In [13]:
response_project = requests.get(
    "https://research-software-directory.org/api/v1/project?select=id,title,description"
    )
response_author = requests.get(
    "https://research-software-directory.org/api/v1/team_member?select=id,project,orcid,given_names,family_names, affiliation"
    )
response_software = requests.get(
    "https://research-software-directory.org/api/v1/software?select=id,concept_doi,brand_name, description"
    )
response_contributor = requests.get(
    "https://research-software-directory.org/api/v1/contributor?select=id,software,orcid,given_names,family_names,affiliation"
    )
    
projects = response_project.json()
authors_proj = response_author.json()
software = response_software.json()
contributor_soft = response_contributor.json()

for a in authors_proj:
    a['name'] = a['given_names'] + ' ' + a['family_names']
    if a['orcid'] == None:
        a['orcid'] = orcid.name_to_orcid_id(a['name'])

In [6]:
RSD = GraphDatabase.driver(uri, auth=(user, password))

In [27]:
def create_project_person(tx, author):
    if author['affiliation'] is None:
        author['affiliation'] = ''
    tx.run("MERGE (p:Person {orcid: $person.orcid, name: $person.name, affiliation: $person.affiliation, worktype: 'project'})",
            person = author)

In [22]:
def get_project_coauthor(rsd_db_path):
    # "/Users/jennifer/RCSNA/data/rsd.db"
    con = sqlite3.connect(rsd_db_path)
    cur = con.cursor()
    res = cur.execute("""
        SELECT GROUP_CONCAT(orcid, ',') AS orcids 
        FROM author 
        GROUP BY project 
        HAVING COUNT(*) > 1
        """)
    coauthor = res.fetchall()

    coauthor_links = []
    for au in coauthor:
        au = au[0].strip('(\')')
        au = au.split(',')
        coauthor_links = coauthor_links+list(combinations(au, 2))
    cur.close()

    weights = []
    for i in coauthor_links:
        weights.append(coauthor_links.count(i))

    return coauthor_links, weights

In [18]:
coauthor_links, weights = get_project_coauthor("/Users/jennifer/RCSNA/data/rsd.db")

In [23]:


# MATCH
#   (a:Person),
#   (b:Person)
# WHERE a.name = 'A' AND b.name = 'B'
# CREATE (a)-[r:RELTYPE {name: a.name + '<->' + b.name}]->(b)
# RETURN type(r), r.name 
def add_project_edge(tx, link, weight):
    tx.run("""
        MATCH 
            (a:Person), 
            (b:Person)
        WHERE a.orcid = $link_a AND b.orcid = $link_b
        MERGE (a)-[r:COAUTHOR {worktype: 'project', weight: $W}]-(b)
        """, 
        link_a=link[0], link_b=link[1], W = weight
    )
    

In [28]:
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    with driver.session(database="neo4j") as session:
        # Create some nodes
        for author in authors_proj:
            session.execute_write(create_project_person, author)

        # Create some relationships
        for i in range(len(coauthor_links)):
            session.execute_write(add_project_edge, coauthor_links[i], weights)

In [29]:
len(coauthor_links)

4502

In [30]:
from collections import Counter
len(Counter(coauthor_links))

3998

In [ ]:
session.close()
driver.close()